# Broadband usage with economic and demographic data

- Contains broadband usage information at the ZCTA (zipcode) level from https://github.com/microsoft/USBroadbandUsagePercentages
- Contains economic and demographic data at the ZCTA level from the Census
- The ZCTA spatial graph is built from the Census TIGER/Line shapefiles for 2020

Prerequisites to run the code:

```yaml
# env.yaml
# install with 'conda env create -f env.yml'
# conda activate broadband
name: broadband
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.11
  - pip
  - pip:
    - matplotlib
    - networkx
    - requests
    - geopandas
    - census
    - us
    - black
    - ipywidgets
    - tqdm
    - pyarrow
```

In [7]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import os
import networkx as nx
from census import Census
from tqdm import tqdm
import tempfile

## Load Broadband Usage Data

In [8]:
tmpdir = tempfile.gettempdir()

In [9]:
#%% download and load csv
url = "https://raw.githubusercontent.com/microsoft/USBroadbandUsagePercentages/master/dataset/broadband_data_zipcode.csv"
tgtpath = f'{tmpdir}/broadband_data_zipcode.csv'
if not os.path.exists(tgtpath):
    r = requests.get(url)
    with open(tgtpath, "wb") as f:
        f.write(r.content)
broadband_df = pd.read_csv(tgtpath)


## Create graph

In [10]:
url = "https://www2.census.gov/geo/tiger/TIGER2020/ZCTA520/tl_2020_us_zcta520.zip"
tgtpath = f'{tmpdir}/tl_2020_us_zcta520.zip'

r = requests.get(url)
with open(tgtpath, "wb") as f:
    f.write(r.content)
gdf = gpd.read_file(tgtpath)

G = nx.Graph()
nodes = gdf["ZCTA5CE20"].values 
G.add_nodes_from(nodes)

for i, geom in tqdm(enumerate(gdf.geometry)):
    # find all other geoms that touches
    nbrs = gdf[gdf.geometry.touches(geom)].geometry
    G.nodes[nodes[i]]["lon"] = geom.centroid.x
    G.nodes[nodes[i]]["lat"] = geom.centroid.y
    for j, geom2 in enumerate(nbrs):
        G.add_edge(nodes[i], nodes[j])

33791it [04:39, 120.69it/s]


## Load Census Data

In [11]:
def get_census_data():
    fields = {
        "B03002_001E": "total_population",
        "B03002_003E": "white",
        "B03002_004E": "black",
        "B03002_006E": "asian",
        "B19013_001E": "median_household_income",
        "B15003_017E": "highschool_graduates",
        "B15003_001E": "total_population_over_18",
        "B23025_005E": "unemployed_population_16_over",
        "B25003_002E": "owner_occupied_housing_units",
        "B25003_003E": "renter_occupied_housing_units",
        "B01001_003E": "age_under_5",
        "B01001_004E": "age_5_9",
        "B01001_005E": "age_10_14",
        "B01001_006E": "age_15_19",
        "B01001_007E": "age_20_24",
        "B17001_002E": "population_below_poverty_level"
    }

    c = Census(os.environ.get("CENSUS_KEY"), year=2020)
    df = c.acs5.state_zipcode(list(fields.keys()), Census.ALL, Census.ALL)
    df = pd.DataFrame(df)
    df = df.rename(columns=fields)
    df = df.rename(columns={"zip code tabulation area": "zcta"})
    df = df.set_index("zcta")

    # Calculate new fields
    df['pct_highschool'] = df['highschool_graduates'] / df['total_population_over_18']
    df.loc[df['median_household_income'] <= 0, 'median_household_income'] = np.nan
    df['log_median_income'] = np.log(df['median_household_income'])
    df.loc[df["total_population"] <= 0, "total_population"] = np.nan
    df['log_total_population'] = np.log(df['total_population'])
    
    # Convert ethnicity counts to percentages
    for ethnicity in ["white", "black", "asian"]:
        df[f'pct_{ethnicity}'] = np.minimum(df[ethnicity].values / df['total_population'].values, 1)

    df['pct_poverty'] = np.minimum(df['population_below_poverty_level'].values / df['total_population'].values, 1)

    # Calculate percentage of population per age group
    age_columns = ['age_under_5', 'age_5_9', 'age_10_14', 'age_15_19', 'age_20_24'] #... add other age groups
    for col in age_columns:
        df[f'pct_{col}'] = np.minimum(df[col].values / df['total_population'].values, 1)

    return df

census_data = get_census_data()

## Merge

In [12]:
final_df = gdf[["ZCTA5CE20", "ALAND20", "AWATER20"]].rename(columns={"ZCTA5CE20": "zcta"}).set_index("zcta")
final_df = pd.merge(final_df, census_data, left_index=True, right_index=True)
final_df["pop_dens"] = 100000 * final_df["total_population"] / final_df["ALAND20"]
broadband = broadband_df.rename(columns={"POSTAL CODE": "zcta"})
broadband["zcta"] = broadband["zcta"].astype(str).str.zfill(5)
broadband = broadband.set_index("zcta")["BROADBAND USAGE"]
final_df = pd.merge(final_df, broadband, left_index=True, right_index=True)

In [13]:
final_df.head()

,ALAND20,AWATER20,total_population,white,black,asian,median_household_income,highschool_graduates,total_population_over_18,unemployed_population_16_over,...,pct_black,pct_asian,pct_poverty,pct_age_under_5,pct_age_5_9,pct_age_10_14,pct_age_15_19,pct_age_20_24,pop_dens,BROADBAND USAGE
zcta,,,,,,,,,,,,,,,,,,,,,
01001,29247431,2127894,16064.0,13967.0,387.0,509.0,66088.0,3245.0,12379.0,362.0,...,0.024091,0.031686,0.080989,0.028822,0.022286,0.033927,0.016559,0.005727,54.924482,0.808
01002,137931897,4280973,30099.0,21114.0,1658.0,3101.0,60540.0,1562.0,14177.0,1305.0,...,0.055085,0.103027,0.196684,0.015615,0.016246,0.018705,0.013821,0.045317,21.821639,0.638
01003,2082670,14001,11588.0,7363.0,481.0,2377.0,NaN,6.0,74.0,948.0,...,0.041508,0.205126,0.002071,0.000000,0.000000,0.000000,0.005437,0.265533,556.401158,1.000
01005,111859191,666424,5166.0,4833.0,103.0,17.0,80060.0,1087.0,3578.0,114.0,...,0.019938,0.003291,0.107240,0.006775,0.012776,0.033488,0.021487,0.012195,4.618306,0.280
01007,142111112,6936467,15080.0,13612.0,112.0,529.0,94812.0,2282.0,10584.0,316.0,...,0.007427,0.035080,0.049536,0.029642,0.025332,0.040385,0.025862,0.017308,10.611415,0.496


## Save

In [14]:
tgtdir = "uploads/dataverse"
os.makedirs(tgtdir, exist_ok=True)
final_df.to_csv(f"{tgtdir}/data_broadband.parquet")
nx.write_graphml(G, f"{tgtdir}/graph_broadband.graphml.gz")

In [15]:
print("Number of nodes:", len(G.nodes))
print("Number of edges:", len(G.edges))

Number of nodes: 33791
Number of edges: 184246


## Var list


In [16]:
final_df.columns

Index(['ALAND20', 'AWATER20', 'total_population', 'white', 'black', 'asian',
       'median_household_income', 'highschool_graduates',
       'total_population_over_18', 'unemployed_population_16_over',
       'owner_occupied_housing_units', 'renter_occupied_housing_units',
       'age_under_5', 'age_5_9', 'age_10_14', 'age_15_19', 'age_20_24',
       'population_below_poverty_level', 'pct_highschool', 'log_median_income',
       'log_total_population', 'pct_white', 'pct_black', 'pct_asian',
       'pct_poverty', 'pct_age_under_5', 'pct_age_5_9', 'pct_age_10_14',
       'pct_age_15_19', 'pct_age_20_24', 'pop_dens', 'BROADBAND USAGE'],
      dtype='object')